In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib
import json
import sys
import pprint

def print_r(text):
    print(json.dumps(text, indent=4, sort_keys=False))

api_key = "4tsbjxvbrwnpcjdsh277csqu"
url = "http://api.walmartlabs.com/v1/search" 
params = {
    "apiKey": api_key,
    "format": "json", # json|xml
    "numItems": 25,
    #"start": 1,
    #"categoryId": "3944_3951_1089430_1230093_1230101"
}
params["query"] = "laptop"

In [2]:
def walmart_search(q):
    params["query"] = q
    walmart_data = requests.get(url, params=params)
    walmart_url = walmart_data.url
    walmart_data = walmart_data.json()
    #print(walmart_url)
    #print(walmart_data)
    #result = {}
    #result["totalResults"] = walmart_data["totalResults"]
    #result["start"] = walmart_data["start"]
    #result["numItems"] = walmart_data["numItems"]
    #result["items"] = walmart_data["items"]
    return walmart_data


In [3]:
params["start"] = 1
query = "TV"
query = "Laptop"
query = "887276189529"
re = walmart_search(query)
if re["numItems"]>0:
    print_r(re)

In [4]:
types_df = pd.read_csv("BestBuy_Data.csv")
types_df.head()

,upc,customerReviewAverage,customerReviewCount,inStoreAvailability,manufacturer,name,onlineAvailability,regularPrice,salePrice
0,884116293927,NaN,NaN,True,Dell,"Dell - 17.3"" Laptop - Intel Core i7 - 16GB Mem...",True,1099.99,1099.99
1,884116293392,NaN,NaN,True,Dell,"Dell - Inspiron 15.6"" Laptop - AMD Ryzen 3 - 4...",True,499.99,499.99
2,719192618893,NaN,NaN,True,LG,"LG - gram 13.3"" Laptop - Intel Core i5 - 8GB M...",True,1099.99,1099.99
3,719192618930,NaN,NaN,True,LG,"LG - gram 14"" Touch-Screen Laptop - Intel Core...",True,1549.99,1549.99
4,719192619395,NaN,NaN,True,LG,"LG - gram 15.6"" Touch-Screen Laptop - Intel Co...",True,1999.99,1999.99


In [7]:
out_file = open("walmart_All_data_file2.csv","w")
all_data =[]
out_header = []
out_header.append("itemId")
out_header.append("parentItemId")
#out_header.append("name")
out_header.append("salePrice")
out_header.append("upc")
out_header.append("categoryPath")
out_header.append("numReviews")
out_header.append("customerRating")
#out_header.append("shortDescription")
out_header.append("thumbnailImage")
out_header.append("mediumImage")
out_header.append("largeImage")
out_header.append("categoryNode")
#out_header.append("offerType")
# Write the cvs header
out_file.write(','.join(out_header))

for index, row in types_df.iterrows():
    query = row["upc"]
    re = walmart_search(query)
    try:
        if re["numItems"]>0:
            #print_r(re)
            print(re["items"][0]["name"])
            for x in re["items"]:
                if "numReviews" not in x:
                    x["numReviews"]=0
                    x["customerRating"]="0.0"
                if "largeImage" not in x:
                    x["largeImage"]="0.0"
                if "salePrice" in x and "upc" in x:
                    if "\\" not in x["name"]:
                        out_body = []
                        item_data = {}
                        item_data["itemId"] = x["itemId"]
                        out_body.append(str(x["itemId"]))
                        item_data["parentItemId"] = x["parentItemId"]
                        out_body.append(str(x["parentItemId"]))
                        #x["name"] = x["name"].encode('UTF-8')
                        try:
                            out_body.append(str(x["name"]))
                            item_data["name"] = x["name"]
                        except (KeyError, IndexError):
                            out_body.append(str("Error with the name"))
                            item_data["name"] = "Error with the name"
                            
                        #out_body.append(x["name"].encode('utf-8'))
                        #out_body.append(str(x["name"]))
                        out_body.append(str(x["salePrice"]))
                        item_data["salePrice"] = x["salePrice"]
                        out_body.append(x["upc"])
                        item_data["upc"] = x["upc"]
                        out_body.append(x["categoryPath"])
                        item_data["categoryPath"] = x["categoryPath"]
                        out_body.append(str(x["numReviews"]))
                        item_data["numReviews"] = x["numReviews"]
                        out_body.append(x["customerRating"])
                        item_data["customerRating"] = x["customerRating"]
                        #out_body.append(x["shortDescription"])
                        out_body.append(x["thumbnailImage"])
                        item_data["thumbnailImage"] = x["thumbnailImage"]
                        out_body.append(x["mediumImage"])
                        item_data["mediumImage"] = x["mediumImage"]
                        out_body.append(x["largeImage"])
                        item_data["largeImage"] = x["largeImage"]
                        out_body.append(x["categoryNode"])
                        item_data["categoryNode"] = x["categoryNode"]
                        #out_body.append(x["offerType"])
                        # Write the cvs header
                        #out_file.write(','.join(out_body))
                        #print_r(out_body)
                        out_file.write("\n"+','.join(out_body))
                        all_data.append(item_data)
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    #print(row["upc"], index)
out_file.close() # Close the new datafile
print("We Finished, but not yet")
results_pd = pd.DataFrame(all_data)
results_pd.head()

Dell - G3 17.3" Gaming Laptop, Intel Core i7-8750H, NVIDIA 1050Ti 4GB GPU, 16GB RAM, 128GB SSD + 1TB Storage, G3779-7927BLK-PUS, Gaming Bundle Included
LG gram 13.3" Intel 8th Gen i5-8250U Ultra-Slim Laptop - 13Z980-U.AAW5U1
LG gram 14.0" Intel 8th Gen i7-8550U Ultra-Slim Touch Laptop - 14Z980-A.AAS7U1
LG 15Z980RAAS9 15Z980-R.AAS9U1 gram 15.6" i7, 16GB, 1TB SSD, Windows 10 Touchscreen Laptop
Acer Spin 1 11.6" Laptop Intel Celeron 2.4 GHz 4GB Ram 32GB Flash Windows 10 | Manufacturer Refurbished
Acer 15.6" Aspire Notebook Intel Core i7 2.70 GHz 12GB Ram 1TB HDD Windows10Home | Manufacturer Refurbished
Acer 15.6" Intel Celeron 1.5GHz 4GB Ram 16GB SSD Chrome OS| FR/ENG Edition | Manufacturer Refurbished
Manufacturer Refurbished Acer CB3-532-C47C 15.6" Chromebook, Chrome OS, Intel Celeron N3060 Processor, 2GB RAM, 16GB Flash Drive
Acer 17.3" Intel Core i7 2.90 GHz 64 GB Ram Windows 10 Home | GX-792-75MV | Manufacturer Refurbished
Refurbished Acer Predator Laptop Intel Core i7 2.8 GHz 32 GB 

LG 75" Class 4K (2160) HDR Smart LED UHD TV w/AI ThinQ - 75UK6570PUB
LG 65" OLED 4K HDR Smart OLED TV w/AI ThinQ OLED65B8PUA
SunbriteTV SB-S-43-4K-BL 43"led Ultra Hdtv 3840x2160 4k 60hz 2-hdmi Hdmi 2.0 Hdcp 2.2 Vga No Spkr
43IN 4K BLACK OUTDOOR LED TV BRAND SOURCE ONLY SB-4374UHD-BL
SAMSUNG 55" Class Curved 4K (2160P) Ultra HD Smart LED TV (UN55MU8500FXZA)
SAMSUNG 55" Class 4K (2160P) Ultra HD Smart LED TV (UN55MU9000FXZA)
Samsung 65" Class Curved 4K (2160P) Ultra HD Smart LED TV (UN65MU8500FXZA)
SAMSUNG 55" Class Curved 4K (2160P) Ultra HD Smart QLED HDR TV (QN55Q8CAMFXZA)
LG 65" Class 4K Ultra HD (2160P) Smart OLED TV (OLED65B7A)
SAMSUNG 58" Class 4K (2160P) Ultra HD Smart LED TV (UN58MU6070)
SAMSUNG 82" Class 4K (2160P) Ultra HD Smart LED TV UN82NU8000FXZA (2018 model)
SAMSUNG 32" Class HD (720P) Smart LED TV (UN32M4500)
SAMSUNG 65" Class 4K (2160P) Ultra HD Smart QLED HDR TV QN65Q6FNAFXZA (2018 Model)
SAMSUNG 75" Class 4K (2160P) Ultra HD Smart QLED HDR TV QN75Q7FN (2018 model)
SAM

,categoryNode,categoryPath,customerRating,itemId,largeImage,mediumImage,name,numReviews,parentItemId,salePrice,thumbnailImage,upc
0,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,0.0,260594775,https://i5.walmartimages.com/asr/96cb0f94-ba75...,https://i5.walmartimages.com/asr/96cb0f94-ba75...,"Dell - G3 17.3"" Gaming Laptop, Intel Core i7-8...",0,260594775,1104.00,https://i5.walmartimages.com/asr/96cb0f94-ba75...,884116293927
1,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,0.0,486905649,https://i5.walmartimages.com/asr/35c9950e-302a...,https://i5.walmartimages.com/asr/35c9950e-302a...,"LG gram 13.3"" Intel 8th Gen i5-8250U Ultra-Sli...",0,486905649,996.99,https://i5.walmartimages.com/asr/35c9950e-302a...,719192618893
2,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,4.2,788821428,https://i5.walmartimages.com/asr/7cb981b5-e0f8...,https://i5.walmartimages.com/asr/7cb981b5-e0f8...,"LG gram 14.0"" Intel 8th Gen i7-8550U Ultra-Sli...",5,788821428,1396.99,https://i5.walmartimages.com/asr/7cb981b5-e0f8...,719192618930
3,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,4.539,654858649,https://i5.walmartimages.com/asr/3ebc392e-c378...,https://i5.walmartimages.com/asr/3ebc392e-c378...,"LG 15Z980RAAS9 15Z980-R.AAS9U1 gram 15.6"" i7, ...",13,654858649,1839.00,https://i5.walmartimages.com/asr/3ebc392e-c378...,719192619395
4,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,0.0,862797804,https://i5.walmartimages.com/asr/03fc34d6-2e08...,https://i5.walmartimages.com/asr/03fc34d6-2e08...,"Acer Spin 1 11.6"" Laptop Intel Celeron 2.4 GHz...",0,862797804,278.44,https://i5.walmartimages.com/asr/03fc34d6-2e08...,841631121746


In [27]:
results_pd = pd.DataFrame(all_data)
results_pd = results_pd[["itemId",
                         "upc",
                         "name",
                         "salePrice",
                         "parentItemId",
                         "categoryNode",
                         "categoryPath",
                         "largeImage",
                         "mediumImage",
                         "customerRating",
                         "numReviews",
                         "thumbnailImage"]]
results_pd.head()
results_pd

,itemId,upc,name,salePrice,parentItemId,categoryNode,categoryPath,largeImage,mediumImage,customerRating,numReviews,thumbnailImage
0,260594775,884116293927,"Dell - G3 17.3"" Gaming Laptop, Intel Core i7-8...",1104.00,260594775,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/96cb0f94-ba75...,https://i5.walmartimages.com/asr/96cb0f94-ba75...,0.0,0,https://i5.walmartimages.com/asr/96cb0f94-ba75...
1,486905649,719192618893,"LG gram 13.3"" Intel 8th Gen i5-8250U Ultra-Sli...",996.99,486905649,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/35c9950e-302a...,https://i5.walmartimages.com/asr/35c9950e-302a...,0.0,0,https://i5.walmartimages.com/asr/35c9950e-302a...
2,788821428,719192618930,"LG gram 14.0"" Intel 8th Gen i7-8550U Ultra-Sli...",1396.99,788821428,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/7cb981b5-e0f8...,https://i5.walmartimages.com/asr/7cb981b5-e0f8...,4.2,5,https://i5.walmartimages.com/asr/7cb981b5-e0f8...
3,654858649,719192619395,"LG 15Z980RAAS9 15Z980-R.AAS9U1 gram 15.6"" i7, ...",1839.00,654858649,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/3ebc392e-c378...,https://i5.walmartimages.com/asr/3ebc392e-c378...,4.539,13,https://i5.walmartimages.com/asr/3ebc392e-c378...
4,862797804,841631121746,"Acer Spin 1 11.6"" Laptop Intel Celeron 2.4 GHz...",278.44,862797804,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/03fc34d6-2e08...,https://i5.walmartimages.com/asr/03fc34d6-2e08...,0.0,0,https://i5.walmartimages.com/asr/03fc34d6-2e08...
5,289065310,841631121418,"Acer 15.6"" Aspire Notebook Intel Core i7 2.70 ...",794.99,289065310,3944_3951_1089430,Electronics/Computers/Laptops/Laptops By Brand...,https://i5.walmartimages.com/asr/2d07492e-dee2...,https://i5.walmartimages.com/asr/2d07492e-dee2...,0.0,0,https://i5.walmartimages.com/asr/2d07492e-dee2...
6,668859863,841631107191,"Acer 15.6"" Intel Celeron 1.5GHz 4GB Ram 16GB S...",198.99,668859863,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/a8cc566d-e8cb...,https://i5.walmartimages.com/asr/a8cc566d-e8cb...,0.0,0,https://i5.walmartimages.com/asr/a8cc566d-e8cb...
7,55429976,841631114311,Manufacturer Refurbished Acer CB3-532-C47C 15....,149.99,55429976,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/738cd228-2d3b...,https://i5.walmartimages.com/asr/738cd228-2d3b...,4.348,23,https://i5.walmartimages.com/asr/738cd228-2d3b...
8,891773860,841631122217,"Acer 17.3"" Intel Core i7 2.90 GHz 64 GB Ram Wi...",2285.99,891773860,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/5c032810-a856...,https://i5.walmartimages.com/asr/5c032810-a856...,0.0,0,https://i5.walmartimages.com/asr/5c032810-a856...
9,443276634,841631121760,Refurbished Acer Predator Laptop Intel Core i7...,2379.29,443276634,3944_3951_1089430,Electronics/Computers/Laptops/Shop Laptops by ...,https://i5.walmartimages.com/asr/b061c4a1-c10f...,https://i5.walmartimages.com/asr/b061c4a1-c10f...,0.0,0,https://i5.walmartimages.com/asr/b061c4a1-c10f...
